# Object Counting using Classical Computer Vision

**Author:** Pranathi Kothapalli  
**Description:** This notebook demonstrates object counting using OpenCV techniques like thresholding, morphological operations, and contour detection.

In [ ]:
import cv2
import numpy as np
import matplotlib.pyplot as plt

# Function to display images in Jupyter
def show_image(title, image, cmap='gray'):
    plt.figure(figsize=(10, 6))
    if cmap == 'gray':
        plt.imshow(image, cmap='gray')
    else:
        plt.imshow(cv2.cvtColor(image, cv2.COLOR_BGR2RGB))
    plt.title(title)
    plt.axis('off')
    plt.show()

## 1. Load Image
Set the `IMAGE_PATH` variable to your input image.

In [ ]:
# CONFIGURATION
IMAGE_PATH = 'path/to/your/image.jpg'  # <--- REPLACE THIS WITH YOUR IMAGE PATH

# Load image
original_img = cv2.imread(IMAGE_PATH)

if original_img is None:
    print(f"Error: Could not load image from {IMAGE_PATH}. Please check the path.")
else:
    show_image("Original Image", original_img, cmap=None)

## 2. Preprocessing
- Convert to Grayscale
- Apply Gaussian Blur to reduce noise

In [ ]:
if original_img is not None:
    # Convert to grayscale
    gray = cv2.cvtColor(original_img, cv2.COLOR_BGR2GRAY)
    
    # Blur
    blur = cv2.GaussianBlur(gray, (11, 11), 0)
    
    show_image("Blurred Grayscale", blur)

## 3. Thresholding & Morphological Operations
- Uses Otsu's thresholding for automatic level detection.
- Applies morphological closing to fill gaps.

In [ ]:
if original_img is not None:
    # Thresholding (Otsu)
    # Note: Depending on the image (dark objects on light bg or vice versa), you might need cv2.THRESH_BINARY_INV
    thresh_val, thresh = cv2.threshold(blur, 0, 255, cv2.THRESH_BINARY_INV + cv2.THRESH_OTSU)
    
    # Morphological operations (Closing to fill holes, Opening to remove noise)
    kernel = np.ones((3, 3), np.uint8)
    closing = cv2.morphologyEx(thresh, cv2.MORPH_CLOSE, kernel, iterations=2)
    opening = cv2.morphologyEx(closing, cv2.MORPH_OPEN, kernel, iterations=2)
    
    processed_img = opening
    show_image("Processed Binary Image", processed_img)

## 4. Count and Visualize
- Find contours.
- Filter out small noise based on area.
- Draw masks.

In [ ]:
if original_img is not None:
    # Find contours
    contours, hierarchy = cv2.findContours(processed_img, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    
    # Configurable area threshold to ignore noise
    MIN_AREA = 50  # Adjust this based on image resolution
    
    valid_contours = []
    for cnt in contours:
        if cv2.contourArea(cnt) > MIN_AREA:
            valid_contours.append(cnt)
            
    count = len(valid_contours)
    
    # Draw contours
    result_img = original_img.copy()
    cv2.drawContours(result_img, valid_contours, -1, (0, 255, 0), 2)  # Green contours
    
    # Overlay masks (semi-transparent)
    mask = np.zeros_like(original_img)
    cv2.drawContours(mask, valid_contours, -1, (0, 0, 255), cv2.FILLED) # Red fill
    
    # Blend original and mask
    alpha = 0.4
    final_output = cv2.addWeighted(result_img, 1, mask, alpha, 0)
    
    print(f"\n------------------------------")
    print(f"DETECTED OBJECTS: {count}")
    print(f"------------------------------\n")
    
    show_image(f"Final Result - Count: {count}", final_output, cmap=None)